# MNIST Visualization Example

Real-time visualization of MNIST training on a CNN, using TensorFlow and [TensorDebugger](https://github.com/ericjang/tdb)

The visualizations in this notebook won't show up on http://nbviewer.ipython.org. To view the widgets and interact with them, you will need to download this notebook and run it with a Jupyter Notebook server.

In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
import tensorflow as tf
import sys
sys.path.append('/home/evjang/thesis/tensor_debugger')
import tdb

## Build TensorFlow Model

In [3]:
from tdb.examples import mnist, viz

In [4]:
(train_data_node,
    train_labels_node,
    validation_data_node,
    test_data_node,
    # predictions
    train_prediction,
    validation_prediction,
    test_prediction,
    # weights
    conv1_weights,
    conv2_weights,
    fc1_weights,
    fc2_weights,
    # training
    optimizer,
    loss,
    learning_rate,
    summaries) = mnist.build_model()

## Attach TDB Plot Ops

In [5]:
g=tf.get_default_graph()
p0=tdb.plot_op(viz.viz_activations,inputs=[train_prediction])
p1=tdb.plot_op(viz.viz_conv_weights,inputs=[g.as_graph_element(conv1_weights)])
p2=tdb.plot_op(viz.viz_conv_weights,inputs=[g.as_graph_element(conv2_weights)])
p3=tdb.plot_op(viz.viz_fc_weights,inputs=[g.as_graph_element(fc1_weights)])
p4=tdb.plot_op(viz.viz_fc_weights,inputs=[g.as_graph_element(fc2_weights)])
p2=tdb.plot_op(viz.viz_conv_hist,inputs=[g.as_graph_element(conv1_weights)])
ploss=tdb.plot_op(viz.watch_loss,inputs=[loss])

## Debug + Visualize!

In [6]:
(train_data, 
 train_labels, 
 validation_data, 
 validation_labels, 
 test_data, 
 test_labels) = mnist.get_data()

('Extracting', '/contrib/projects/tensorflow/tensorflow/tensorflow/models/image/mnist/data/train-images-idx3-ubyte.gz')
('Extracting', '/contrib/projects/tensorflow/tensorflow/tensorflow/models/image/mnist/data/train-labels-idx1-ubyte.gz')
('Extracting', '/contrib/projects/tensorflow/tensorflow/tensorflow/models/image/mnist/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/contrib/projects/tensorflow/tensorflow/tensorflow/models/image/mnist/data/t10k-labels-idx1-ubyte.gz')


In [ ]:
s=tf.InteractiveSession()
tf.initialize_all_variables().run()

In [ ]:
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 5
TEST_SIZE=55000
TRAIN_SIZE=10000

for step in xrange(NUM_EPOCHS * TRAIN_SIZE // BATCH_SIZE):
    offset = (step * BATCH_SIZE) % (TRAIN_SIZE - BATCH_SIZE)
    batch_data = train_data[offset:(offset + BATCH_SIZE), :, :, :]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    feed_dict = {
        train_data_node: batch_data,
        train_labels_node: batch_labels
    }
    # run training node and visualization node
    status,result=tdb.debug([optimizer,p0], feed_dict=feed_dict, session=s)
    if step % 10 == 0:        
        status,result=tdb.debug([loss,p1,p2,p3,p4,ploss], feed_dict=feed_dict, session=s)
        print('loss: %f' % (result[0]))

loss: 29.668428
loss: 15.983353
loss: 11.249242
loss: 10.028939
loss: 8.065391
loss: 9.335689
loss: 7.316875
loss: 8.376289
loss: 7.735221
loss: 8.383675
loss: 5.704120
loss: 6.037778
loss: 7.309663
loss: 7.349874
loss: 7.528041
loss: 8.209503
loss: 5.364178
loss: 6.238027
loss: 4.885421
loss: 7.149874
loss: 6.667716
loss: 7.664586